## **앱스토어 리뷰 감성 분석**

### 문제정의
각 앱들의 리뷰들과 평점이 양의 관계에 있다고 가정하고 학습을 진행하여 평점을 예측

### 데이터 불러오기

In [3]:
import ujson
with open('./data/train_data_mecab.txt', 'r') as f:
    text = f.read()

train_docs = ujson.loads(text)

with open('./data/test_data_mecab.txt', 'r') as f:
    text = f.read()

test_docs = ujson.loads(text)

In [4]:
len(train_docs), len(test_docs)

(489847, 126911)

In [1]:
def get_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f.readlines()]
        docs = [doc for doc in docs if len(doc) == 3]
        
    if not docs:
        app_ids, texts, ratings = [], [], []

    app_ids, texts, ratings = zip(*docs)
    return list(app_ids), list(texts), list(ratings)

In [2]:
train_app_ids, train_texts, train_ratings = get_data('./data/train_docs.txt')
test_app_ids, test_texts, test_ratings = get_data('./data/test_docs.txt')

띄어쓰기 문제: 웹에서 수집한 다양한 데이터를 가지고 우선 모델을 training 시킨다. 트위터 api 웹 크롤링등 이 좋겠군

In [38]:
# with open('./data/train_texts.txt', 'w', encoding='utf-8') as f:
#     for doc in train_texts:
#         print(doc, file=f)

In [18]:
from soyspacing.countbase import CountSpace, RuleDict

In [22]:
model = CountSpace()
model.train('./data/train_texts.txt')

all tags length = 1497385 --> 469716, (num_doc = 489846)

In [23]:
model_path = './spacing_model/sp_model.txt'
model.save_model(model_path, json_format=False)

In [21]:
model = CountSpace()
model_path = './spacing_model/sp_model.txt'
model.load_model(model_path)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [64]:
no_blank = []
for i, doc in enumerate(test_texts):
    if not ' ' in doc:
        no_blank.append(i)

In [91]:
idx = np.random.choice(no_blank)
sent = test_texts[idx]
sent

'왜결제가되지?????장난치나ㅡㅡ'

In [92]:
verbose=False
mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold

sent_corrected, tags = model.correct(doc=sent, verbose=verbose, force_abs_threshold=ft, nonspace_threshold=nt, space_threshold=st, min_count=mc)

print('before: %s' % sent)
print('after : %s' % sent_corrected)

before: 왜결제가되지?????장난치나ㅡㅡ
after : 왜 결제가 되지?????장난치나ㅡㅡ


### 데이터 전처리
1. 축약어, 오타:
골치거리임, 전형적인 한국 사전에 없는 단어들도 많음, 사전에 없는 단어를 트위터사전에 넣어야 할 듯
2. 띄어쓰기가 불분명함

$\Rightarrow$ data Cleaning class 필요성 있음

### 데이터 후처리

데이터 후처리란 형태소 분석기를 거친 후에 이상한 단어들, 신조어 등을 사전에 추가하는 작업


### Exploration

단어장을 만듬, 하지만 형태소 분석이 안된건 다시 분석해야함

In [38]:
from konlpy.tag import Twitter, Kkma, Komoran, Mecab

In [39]:
twitter = Twitter()
kkma = Kkma()
komo = Komoran()
mecab = Mecab()

In [8]:
text = '구매내역 삭제부탁드려요ㅠㅠ'
for tokenizer in [twitter, kkma, komo, mecab]:
    print(tokenizer.pos(text))

[('구매', 'Noun'), ('내', 'Determiner'), ('역', 'Noun'), ('삭제', 'Noun'), ('부탁드려', 'Adjective'), ('요', 'Eomi'), ('ㅠㅠ', 'KoreanParticle')]
[('구매', 'NNG'), ('내역', 'NNG'), ('삭제', 'NNG'), ('부탁드리', 'VV'), ('어요', 'EFN'), ('ㅠㅠ', 'EMO')]
[('구매', 'NNG'), ('내역', 'NNG'), ('삭제부탁드려요ㅠㅠ', 'NA')]
[('구매', 'NNG'), ('내역', 'NNG'), ('삭제', 'NNG'), ('부탁드려요', 'VV+EF'), ('ㅠㅠ', 'UNKNOWN')]
